## Train image classifier on dogs cats dataset

### Import libraries

In [ ]:
import os
import time
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn.functional as F
import torch.nn as nn
import torchvision

from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torchvision import transforms
from torchvision.datasets import ImageFolder
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.metrics import roc_auc_score
from PIL import Image

%matplotlib inline
if torch.cuda.is_available():
    torch.backends.cudnn.deterministic = True

### Setup log folder

In [ ]:
EXPERIMENT_NAME = "cnn_dogscats" 
ROOT_DIR = os.path.abspath(".")
LOG_PATH = os.path.join(ROOT_DIR, "logs", EXPERIMENT_NAME)

if not os.path.exists(os.path.join(ROOT_DIR, "logs")):
    os.mkdir(os.path.join(ROOT_DIR, "logs"))
    
if not os.path.exists(LOG_PATH):
    os.mkdir(LOG_PATH)

### Setup params

In [ ]:
# Device
DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(DEVICE)

# Hyperparameters
RANDOM_SEED = 1
LEARNING_RATE = 0.001
NUM_EPOCHS = 10
BATCH_SIZE = 32

# Architecture
NUM_CLASSES = 2

### Download data and setup path

In [ ]:
# Get data, use if only data is not available already
# os.system("mkdir ./datasets")
# os.system("wget -O ./datasets/dogscats.zip https://github.com/hasibzunair/my-releases/releases/download/v1-datasets/dogscats.zip")
# os.system("unzip ./datasets/dogscats.zip -d ./datasets/")

# Setup training data
TRAIN_ROOT = './datasets/dogscats/train'
num_train_cats = len([i for i in os.listdir(os.path.join(TRAIN_ROOT, 'Cat')) 
                      if i.endswith('.jpg') and i.startswith('cat')])

num_train_dogs = len([i for i in os.listdir(os.path.join(TRAIN_ROOT, 'Dog')) 
                      if i.endswith('.jpg') and i.startswith('dog')])

print(f'Training set cats: {num_train_cats}')
print(f'Training set dogs: {num_train_dogs}')

# Setup validation data
VAL_ROOT = './datasets/dogscats/val'
num_train_cats = len([i for i in os.listdir(os.path.join(VAL_ROOT, 'Cat')) 
                      if i.endswith('.jpg') and i.startswith('cat')])

num_train_dogs = len([i for i in os.listdir(os.path.join(VAL_ROOT, 'Dog')) 
                      if i.endswith('.jpg') and i.startswith('dog')])

print(f'Validation set cats: {num_train_cats}')
print(f'Validation set dogs: {num_train_dogs}')

### Plot and image

In [ ]:
img = Image.open(os.path.join(TRAIN_ROOT, 'Cat', 'cat.59.jpg'))
print(np.asarray(img, dtype=np.uint8).shape)
plt.imshow(img);

In [ ]:
img = Image.open(os.path.join(TRAIN_ROOT, 'Dog', 'dog.23.jpg'))
print(np.asarray(img, dtype=np.uint8).shape)
plt.imshow(img);

### Applying Imagefolder Class and Transforms


In [ ]:
## setting directories
data_dir = './datasets/dogscats/'
train_dir = os.path.join(data_dir, 'train') 
val_dir = os.path.join(data_dir, 'val')

In [ ]:
## transform
train_transforms = transforms.Compose([
    transforms.Resize((224, 224)), # resize image
    transforms.CenterCrop((224, 224)), # center crop image
    transforms.ToTensor(), # convert to Pytorch tensor
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)) # normalize
])

val_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.CenterCrop((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

train_dataset = ImageFolder(train_dir, train_transforms)
val_dataset = ImageFolder(val_dir, val_transforms)
test_dataset = ImageFolder(val_dir, val_transforms) # note: val and test is same

print(f'Number of training examples: {len(train_dataset)}')
print(f'Number of validation examples: {len(val_dataset)}')
print(f'Number of test examples: {len(test_dataset)}')

In [ ]:
## layout of the dataset

print(train_dataset.class_to_idx)
print(val_dataset.class_to_idx)
print(test_dataset.class_to_idx)

### Setup dataloader

In [ ]:
# train loader and test loader
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, drop_last=True,
                          shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE,
                          shuffle=False)
test_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE,
                          shuffle=False)

### Visualize data from data loader

In [ ]:
def to_img(ten):
    # Un-normalize and convert to RGB format for visualization
    ten =(ten[0].permute(1,2,0).detach().cpu().numpy()+1)/2
    ten=(ten*255).astype(np.uint8)
    return ten

In [ ]:
cat = train_dataset[1] ## cat is tuple
dog = train_dataset[1500] ## dog is a tuple, dog[0] holds its image tensor

In [ ]:
a = to_img(cat)
print(a.shape)
plt.imshow(a)

In [ ]:
a = to_img(dog)
print(a.shape)
plt.imshow(a)

### Define ConvNet model

In [ ]:
##########################
### MODEL
##########################


class VGG16(torch.nn.Module):

    def __init__(self, num_classes):
        super(VGG16, self).__init__()
        self.block_1 = nn.Sequential(
                nn.Conv2d(in_channels=3,
                          out_channels=64,
                          kernel_size=(3, 3),
                          stride=(1, 1),
                          # (1(32-1)- 32 + 3)/2 = 1
                          padding=1), 
                nn.ReLU(),
                nn.Conv2d(in_channels=64,
                          out_channels=64,
                          kernel_size=(3, 3),
                          stride=(1, 1),
                          padding=1),
                nn.ReLU(),
                nn.MaxPool2d(kernel_size=(2, 2),
                             stride=(2, 2))
        )
        
        self.block_2 = nn.Sequential(
                nn.Conv2d(in_channels=64,
                          out_channels=128,
                          kernel_size=(3, 3),
                          stride=(1, 1),
                          padding=1),
                nn.ReLU(),
                nn.Conv2d(in_channels=128,
                          out_channels=128,
                          kernel_size=(3, 3),
                          stride=(1, 1),
                          padding=1),
                nn.ReLU(),
                nn.MaxPool2d(kernel_size=(2, 2),
                             stride=(2, 2))
        )
        
        self.block_3 = nn.Sequential(        
                nn.Conv2d(in_channels=128,
                          out_channels=256,
                          kernel_size=(3, 3),
                          stride=(1, 1),
                          padding=1),
                nn.ReLU(),
                nn.Conv2d(in_channels=256,
                          out_channels=256,
                          kernel_size=(3, 3),
                          stride=(1, 1),
                          padding=1),
                nn.ReLU(),        
                nn.Conv2d(in_channels=256,
                          out_channels=256,
                          kernel_size=(3, 3),
                          stride=(1, 1),
                          padding=1),
                nn.ReLU(),
                nn.Conv2d(in_channels=256,
                          out_channels=256,
                          kernel_size=(3, 3),
                          stride=(1, 1),
                          padding=1),
                nn.ReLU(),
                nn.MaxPool2d(kernel_size=(2, 2),
                             stride=(2, 2))
        )
        
          
        self.block_4 = nn.Sequential(   
                nn.Conv2d(in_channels=256,
                          out_channels=512,
                          kernel_size=(3, 3),
                          stride=(1, 1),
                          padding=1),
                nn.ReLU(),        
                nn.Conv2d(in_channels=512,
                          out_channels=512,
                          kernel_size=(3, 3),
                          stride=(1, 1),
                          padding=1),
                nn.ReLU(),        
                nn.Conv2d(in_channels=512,
                          out_channels=512,
                          kernel_size=(3, 3),
                          stride=(1, 1),
                          padding=1),
                nn.ReLU(),
                nn.Conv2d(in_channels=512,
                          out_channels=512,
                          kernel_size=(3, 3),
                          stride=(1, 1),
                          padding=1),
                nn.ReLU(),   
                nn.MaxPool2d(kernel_size=(2, 2),
                             stride=(2, 2))
        )
        
        self.block_5 = nn.Sequential(
                nn.Conv2d(in_channels=512,
                          out_channels=512,
                          kernel_size=(3, 3),
                          stride=(1, 1),
                          padding=1),
                nn.ReLU(),            
                nn.Conv2d(in_channels=512,
                          out_channels=512,
                          kernel_size=(3, 3),
                          stride=(1, 1),
                          padding=1),
                nn.ReLU(),            
                nn.Conv2d(in_channels=512,
                          out_channels=512,
                          kernel_size=(3, 3),
                          stride=(1, 1),
                          padding=1),
                nn.ReLU(),
                nn.Conv2d(in_channels=512,
                          out_channels=512,
                          kernel_size=(3, 3),
                          stride=(1, 1),
                          padding=1),
                nn.ReLU(),   
                nn.MaxPool2d(kernel_size=(2, 2),
                             stride=(2, 2))             
        )
        
        self.classifier = nn.Sequential(
                nn.Linear(512*7*7, 4096),
                nn.ReLU(),   
                nn.Linear(4096, 4096),
                nn.ReLU(),
                nn.Linear(4096, num_classes)
        )
            
        
        for m in self.modules():
            if isinstance(m, torch.nn.Conv2d):
                m.weight.detach().normal_(0, 0.05)
                if m.bias is not None:
                    m.bias.detach().zero_()
            elif isinstance(m, torch.nn.Linear):
                m.weight.detach().normal_(0, 0.05)
                m.bias.detach().detach().zero_()
        
        
    def forward(self, x):
        x = self.block_1(x)
        x = self.block_2(x)
        x = self.block_3(x)
        x = self.block_4(x)
        x = self.block_5(x)
        logits = self.classifier(x.view(-1, 512*7*7))
        probas = F.softmax(logits, dim=1)
        return logits, probas

### Build model and define optimizer

In [ ]:
torch.manual_seed(RANDOM_SEED)
model = VGG16(num_classes=NUM_CLASSES)
model = model.to(DEVICE)
print(model)

optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)

### Train

In [ ]:
def compute_accuracy_and_loss(model, data_loader, device):
    correct_pred, num_examples = 0, 0
    cross_entropy = 0.
    for i, (features, targets) in enumerate(data_loader):
            
        features = features.to(device)
        targets = targets.to(device)

        logits, probas = model(features)
        cross_entropy += F.cross_entropy(logits, targets).item()
        _, predicted_labels = torch.max(probas, 1)
        num_examples += targets.size(0)
        correct_pred += (predicted_labels == targets).sum()
    return correct_pred.float()/num_examples * 100, cross_entropy/num_examples
    

start_time = time.time()
train_acc_lst, valid_acc_lst = [], []
train_loss_lst, valid_loss_lst = [], []
best_acc = 0

for epoch in range(NUM_EPOCHS):
    
    model.train()
    
    for batch_idx, (features, targets) in enumerate(train_loader):
    
        ### PREPARE MINIBATCH
        features = features.to(DEVICE)
        targets = targets.to(DEVICE)
            
        ### FORWARD AND BACK PROP
        logits, probas = model(features)
        cost = F.cross_entropy(logits, targets)
        optimizer.zero_grad()
        
        cost.backward()
        
        ### UPDATE MODEL PARAMETERS
        optimizer.step()
        
        ### LOGGING, every 100 batches
        if not batch_idx % 25: 
            print (f'Epoch: {epoch+1:03d}/{NUM_EPOCHS:03d} | '
                   f'Batch {batch_idx:03d}/{len(train_loader):03d} |' 
                   f' Cost: {cost:.4f}')

    # no need to build the computation graph for backprop when computing accuracy
    model.eval()
    with torch.set_grad_enabled(False):
        train_acc, train_loss = compute_accuracy_and_loss(model, train_loader, device=DEVICE)
        valid_acc, valid_loss = compute_accuracy_and_loss(model, val_loader, device=DEVICE)
        
        #Save best model
        if valid_acc > best_acc:
            print("Saving model at accuracy={:.3f}".format(valid_acc))
            torch.save(model.state_dict(), '{}/{}.pth'.format(LOG_PATH, 'model'))
            best_acc = valid_acc
            
        train_acc_lst.append(train_acc)
        valid_acc_lst.append(valid_acc)
        train_loss_lst.append(train_loss)
        valid_loss_lst.append(valid_loss)
        print(f'Epoch: {epoch+1:03d}/{NUM_EPOCHS:03d} Train Acc.: {train_acc:.2f}%'
              f' | Validation Acc.: {valid_acc:.2f}%')
        
    elapsed = (time.time() - start_time)/60
    print(f'Time elapsed: {elapsed:.2f} min')

elapsed = (time.time() - start_time)/60
print(f'Total Training Time: {elapsed:.2f} min') # 20 min per epoch

### Plot loss and accuracy

In [ ]:
new_train_acc_lst, new_valid_acc_lst = [], []
for i in train_acc_lst:
    i = i.cpu()
    new_train_acc_lst.append(i)
for l in valid_acc_lst:
    l = l.cpu()
    new_valid_acc_lst.append(l)

In [ ]:
train_loss_lst = np.array(train_loss_lst)
np.savetxt("{}/{}_train_loss.txt".format(LOG_PATH, EXPERIMENT_NAME), train_loss_lst, delimiter=",")
valid_loss_lst = np.array(valid_loss_lst)
np.savetxt("{}/{}_valid_loss.txt".format(LOG_PATH, EXPERIMENT_NAME), valid_loss_lst, delimiter=",")

train_acc_lst = np.array(new_train_acc_lst)
np.savetxt("{}/{}_train_acc.txt".format(LOG_PATH, EXPERIMENT_NAME), train_acc_lst, delimiter=",")
valid_acc_lst = np.array(new_valid_acc_lst)
np.savetxt("{}/{}_valid_acc.txt".format(LOG_PATH, EXPERIMENT_NAME), valid_acc_lst, delimiter=",")

plt.plot(range(1, NUM_EPOCHS+1), train_loss_lst, label='Training loss')
plt.plot(range(1, NUM_EPOCHS+1), valid_loss_lst, label='Validation loss')
plt.legend(loc='upper right')
plt.ylabel('Cross entropy')
plt.xlabel('Epoch')
plt.savefig('{}/{}_loss_graph.png'.format(LOG_PATH, EXPERIMENT_NAME), dpi=300)
plt.show()

In [ ]:
plt.plot(range(1, NUM_EPOCHS+1), new_train_acc_lst, label='Training accuracy')
plt.plot(range(1, NUM_EPOCHS+1), new_valid_acc_lst, label='Validation accuracy')
plt.legend(loc='upper left')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.savefig('{}/{}_acc_graph.png'.format(LOG_PATH, EXPERIMENT_NAME), dpi=300)
plt.show()

### Testing

In [ ]:
model.eval()
with torch.set_grad_enabled(False): # save memory during inference
    test_acc, test_loss = compute_accuracy_and_loss(model, val_loader, DEVICE)
    print(f'Test accuracy: {test_acc:.2f}%')

In [ ]:
class UnNormalize(object):
    def __init__(self, mean, std):
        self.mean = mean
        self.std = std

    def __call__(self, tensor):
        """
        Args:
            tensor (Tensor): Tensor image of size (C, H, W) to be normalized.
        Returns:
            Tensor: Normalized image.
        """
        for t, m, s in zip(tensor, self.mean, self.std):
            t.mul_(s).add_(m)
            # The normalize code -> t.sub_(m).div_(s)
        return tensor
    
    
train_mean, train_std = [0.5, 0.5, 0.5], [0.5, 0.5, 0.5]

train_mean = torch.tensor(train_mean)
train_std = torch.tensor(train_std)

print(train_mean)
print(train_std)

unorm = UnNormalize(mean=train_mean, std=train_std)

### Visualize some predictions 

In [ ]:
val_loader = DataLoader(dataset=train_dataset, 
                         batch_size=BATCH_SIZE, 
                         shuffle=True)

for features, targets in val_loader:
    break
    

_, predictions = model.forward(features[:8].to(DEVICE))
predictions = torch.argmax(predictions, dim=1)

d = {0: 'cat',
     1: 'dog'}
    
fig, ax = plt.subplots(1, 8, figsize=(20, 10))
for i in range(8):
    img = unorm(features[i])
    ax[i].imshow(np.transpose(img, (1, 2, 0)))
    ax[i].set_xlabel(d[predictions[i].item()])

plt.show()

In [ ]:
test_loader = DataLoader(dataset=val_dataset, 
                         batch_size=BATCH_SIZE, 
                         shuffle=True)

for features, targets in test_loader:
    break
    

_, predictions = model.forward(features[:8].to(DEVICE))
predictions = torch.argmax(predictions, dim=1)

d = {0: 'cat',
     1: 'dog'}
    
fig, ax = plt.subplots(1, 8, figsize=(20, 10))
for i in range(8):
    img = unorm(features[i])
    ax[i].imshow(np.transpose(img, (1, 2, 0)))
    ax[i].set_xlabel(d[predictions[i].item()])

plt.show()

### Infer on one input image

In [ ]:
# Load model
torch.manual_seed(RANDOM_SEED)
model_path = f'./logs/{EXPERIMENT_NAME}/model.pth'
model = VGG16(num_classes=NUM_CLASSES)
checkpoint = torch.load(model_path)
model.load_state_dict(checkpoint)
model = model.to(DEVICE)
model.eval()

In [ ]:
# Inference!
def inference(input_image_path):
    image = Image.open(input_image_path).convert("RGB")
    class_index = {0: 'cat', 1: 'dog'}

    preprocess = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.CenterCrop((224, 224)),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
    ])

    input_tensor = preprocess(image)
    input_batch = input_tensor.unsqueeze(0) # create a mini-batch as expected by the model

    # Move the input and model to GPU for speed if available
    if torch.cuda.is_available():
        input_batch = input_batch.to('cuda')
        model.to('cuda')

    with torch.no_grad():
        _, predictions = model.forward(input_batch)
        
    predictions = torch.argmax(predictions, dim=1)
    label = class_index[predictions.item()]
    return label

In [ ]:
path_to_img = os.path.join(TRAIN_ROOT, 'Dog', 'dog.25.jpg')
#path_to_img = os.path.join(TRAIN_ROOT, 'Cat', 'cat.20.jpg')

prediction = inference(path_to_img)
print(f"The predicted label is {prediction}.")

img = Image.open(path_to_img)
print(np.asarray(img, dtype=np.uint8).shape)
plt.imshow(img)

### Evaluate on various metrics

In [ ]:
print(len(test_dataset))
print(len(test_loader))
print(test_dataset.class_to_idx)

In [ ]:
def evaluation(model, data_loader, device):
    correct_pred, num_examples = 0, 0
    cross_entropy = 0.
    predictions = []
    tgts = []
    for i, (features, targets) in enumerate(data_loader):
            
        features = features.to(device)
        targets = targets.to(device)

        logits, probas = model(features)
        cross_entropy += F.cross_entropy(logits, targets).item()
        _, predicted_labels = torch.max(probas, 1)
        num_examples += targets.size(0)
        correct_pred += (predicted_labels == targets).sum()
        predicted_labels = predicted_labels.detach().cpu().numpy()
        predictions.extend(predicted_labels)
        targets = targets.detach().cpu().numpy()
        tgts.extend(targets)
    return correct_pred.float()/num_examples * 100, cross_entropy/num_examples, tgts, predictions

### Load trained model

In [ ]:
## load best model
torch.manual_seed(RANDOM_SEED)

model_path = f'./logs/{EXPERIMENT_NAME}/model.pth'
model = VGG16(num_classes=NUM_CLASSES)
checkpoint = torch.load(model_path)
model.load_state_dict(checkpoint)
model = model.to(DEVICE)
model.eval()

### Get accuracy, precision, recall, AUC etc.

In [ ]:
with torch.set_grad_enabled(False): # save memory during inference
    test_acc, test_loss, y_true, y_pred = evaluation(model, test_loader, DEVICE)
    print(f'Test accuracy: {test_acc:.2f}%')

In [ ]:
print(classification_report(y_true, y_pred))

In [ ]:
print(f'area under roc curve = {roc_auc_score(y_true, y_pred)}')

In [ ]:
tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()

print(f'TP = {tp}')
print(f'FN = {fn}')
print(f'FP = {fp}')
print(f'TN = {tn}')

sens = tp/(tp+fn) * 100 
specfs = tn/(tn+fp) * 100 

print(10*'=')
print(f'Sensitivity = {sens}%')
print(f'Specificity = {specfs}%')


### Acknowledgements

Parts of this notebook were written by https://github.com/mahmudhasankhan.